In [1]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import matplotlib.pyplot as plt
tf.compat.v1.set_random_seed(0)
from tensorflow import keras
import numpy as np
np.random.seed(0)
import itertools
from keras.preprocessing.image import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from sklearn.metrics import precision_score, accuracy_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

<h1>Data Loading</h1>
<p>Setting up Image Data Generators<p>

In [2]:
#train_gen = image_dataset_from_directory(directory="../input/new-plant-diseases-dataset/train",image_size=(256, 256))
#test_gen = image_dataset_from_directory(directory="../input/new-plant-diseases-dataset/valid",image_size=(256, 256))

train_gen = image_dataset_from_directory(directory="../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",
                                         image_size=(256, 256))
test_gen = image_dataset_from_directory(directory="../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid",
                                        image_size=(256, 256))

rescale = Rescaling(scale=1.0/255)
train_gen = train_gen.map(lambda image,label:(rescale(image),label))
test_gen  = test_gen.map(lambda image,label:(rescale(image),label))

Found 70295 files belonging to 38 classes.
Found 17572 files belonging to 38 classes.


<h1>Modelling</h1>

In [14]:
from tensorflow import keras
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import layers, models

# Load pretrained DenseNet121 as the base (exclude the fully connected top layers)
base_model = DenseNet121(
    weights='imagenet',        # pretrained on ImageNet
    include_top=False,         # drop the original FC layers
    input_shape=(256, 256, 3)  # your input shape
)

# Freeze the base model initially
base_model.trainable = False

# Build the model
model = models.Sequential([
    base_model,                          # feature extractor
    layers.GlobalAveragePooling2D(),     # better than Flatten for DenseNet
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(38, activation='softmax')  # 38 classes
])

# Compile
opt = keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


29097984/29084464 [==============================] - 0s 0us/step
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 8, 8, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 38)                19494     
Total params: 7,581,798
Trainable params: 544,294
Non-trainable params: 7,037,504
_________________________________________________________________


In [15]:
ep = 10
history = model.fit_generator(train_gen,
          validation_data=test_gen,
          epochs = ep)

Epoch 1/10
2197/2197 [==============================] - 200s 89ms/step - loss: 1.0986 - accuracy: 0.7074 - val_loss: 0.3388 - val_accuracy: 0.9202
Epoch 2/10
2197/2197 [==============================] - 192s 88ms/step - loss: 0.3641 - accuracy: 0.8916 - val_loss: 0.2050 - val_accuracy: 0.9467
Epoch 3/10
2197/2197 [==============================] - 194s 88ms/step - loss: 0.2563 - accuracy: 0.9213 - val_loss: 0.1585 - val_accuracy: 0.9553
Epoch 4/10
2197/2197 [==============================] - 196s 89ms/step - loss: 0.1989 - accuracy: 0.9381 - val_loss: 0.1301 - val_accuracy: 0.9633
Epoch 5/10
2197/2197 [==============================] - 196s 89ms/step - loss: 0.1670 - accuracy: 0.9483 - val_loss: 0.1144 - val_accuracy: 0.9669
Epoch 6/10
2197/2197 [==============================] - 195s 89ms/step - loss: 0.1432 - accuracy: 0.9554 - val_loss: 0.1004 - val_accuracy: 0.9685
Epoch 7/10
2197/2197 [==============================] - 195s 89ms/step - loss: 0.1265 - accuracy: 0.9605 - val_loss: 0

<h1>Metrics</h1>

In [16]:
labels = []
predictions = []
for x,y in test_gen:
    labels.append(list(y.numpy()))
    predictions.append(tf.argmax(model.predict(x),1).numpy())

In [17]:
predictions = list(itertools.chain.from_iterable(predictions))
labels = list(itertools.chain.from_iterable(labels))

In [18]:
print("Train Accuracy  : {:.2f} %".format(history.history['accuracy'][-1]*100))
print("Test Accuracy   : {:.2f} %".format(accuracy_score(labels, predictions) * 100))
print("Precision Score : {:.2f} %".format(precision_score(labels, predictions, average='micro') * 100))
print("Recall Score    : {:.2f} %".format(recall_score(labels, predictions, average='micro') * 100))

Train Accuracy  : 96.99 %
Test Accuracy   : 97.37 %
Precision Score : 97.37 %
Recall Score    : 97.37 %
